# sPyTial for Z3

Traditional Z3 work is purely textual - you write constraints, get satisfiable/unsatisfiable results, and examine models through print statements. sPyTial transforms this by making the structure of your constraints, models, and solving process spatially visible.


In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from z3 import *
from spytial import diagram, orientation, group, atomColor, attribute




Let's start with a basic problem and compare traditional vs sPyTial approaches.

In [2]:
# Traditional Z3 approach
x, y = Ints('x y')
solver = Solver()
solver.add(x + y == 10, x > 0, y > 0, x < 8)

print(f"Constraints: {solver.assertions()}")
result = solver.check()
print(f"Result: {result}")

if result == sat:
    model = solver.model()
    print(f"Model: {model}")
    print(f"x = {model[x]}, y = {model[y]}")
else:
    print("No solution found")
    

Constraints: [x + y == 10, x > 0, y > 0, x < 8]
Result: sat
Model: [y = 3, x = 7]
x = 7, y = 3


In [6]:
# sPyTial approach - visualize the model structure.
## THis is actually far less


if result == sat:
    model = solver.model()
    print("Z3 Model Structure:")

    # This is really not a great visualization of the model
    diagram(model, method="inline")


Z3 Model Structure:


## Domain Specific Customization

Since `sPyTial` doesn't know *how* to relationalize a `ModelObj`, we register a relationalizer that does so.


### Shortcomings & trade-offs of this relationalizer.

While relationalizers can be non-trivial, they really need only be implemented once by a domain expert to enable all subsequent spytial diagrams.


*This relationalizer (though large) does a reasonable job for demonstration purposes, but one could imagine an even more sophisticated one for Z3 models*.






- No totalization by default
We only emit explicit function entries and array stores (plus default). If a function’s domain is finite (uninterpreted sorts with universes) and you want a total graph, you’d need to cross-product universes and call m.eval(f(*args), model_completion=True). That’s omitted here to avoid blow-ups; add it behind a “safe size” guard if you need it.

- Datatype selector edges are minimal.
Z3’s Python API around datatypes/accessors can be version-fragile. The code leaves selector emission commented—easy to enable, but you should test against your Z3 version.

- Arrays with symbolic defaults
We detect a single constant default via is_const_array/is_k. If the default is itself complex or depends on variables (rare in models), we just attach it as a label/value atom; we don’t expand lambdas/as-array metaviews further.

- Overloaded names
Decl atoms are qualified by arity (Decl::<name>/<arity>). If you have true overloads with same name/arity but different sorts, collide-proofing would require adding sort signatures (e.g., f/2[Int×Int→Int]).

- Identity and sharing
We memo by (value.hash(), sort.name()) (fallback (str(value), sort.name())). This keeps identical AstRefs unified. If your model reuses equal-but-distinct values that print identically but should be different nodes, you’ll need a different identity policy.

- Large models / performance
We avoid enumerating anything implicit. If you enable totalization or deep datatype traversal, you must set domain caps (e.g., “max 200 tuples/relation”) to keep the graph usable.

- Strings/seqs escaping
We only newline-escape labels. If your renderer needs stricter escaping (quotes, RTL, etc.), extend _escape_label.



In [13]:


from typing import Any, List, Tuple, Optional
import z3
from spytial import RelationalizerBase, Atom, Relation, relationalizer
import itertools

@relationalizer(priority=100)
class Z3ModelRelationalizer(RelationalizerBase):
    """
    Relationalizer for z3.ModelRef.

    Encoding choices (all binary edges, so they fit your Relation schema):
      • Each declaration (func/const) becomes a Decl-atom ("Decl::<name>/<arity>")
      • Each constant also gets a Const-atom ("Const::<name>") linked by 'defines' to its Decl
      • Constant values:  Const::<name> --value_of--> <value-atom>
      • n-ary function entries: use an App-atom "App::<name>/<entry-index>" with edges:
            App --of--> Decl
            App --arg_i--> <arg-atom>   for each i
            App --result--> <result-atom>
        and, if present, Decl --default--> <else-value-atom>
      • Arrays-as-values on constants are decomposed into Store entries:
            App::<name>_map/<i> --of--> Const::<name>
            App ... --key--> <key-atom>
            App ... --val--> <val-atom>
        plus Const::<name> --<name>_default--> <default-atom>

    Type thunks:
      • We thread the *expected sort* (domain/range/array sorts) into value→atom creation.
        This ensures numerals like “3” become Ints when y():Int = 3, etc.
    """

    # ---------- Public API ----------

    def can_handle(self, obj: Any) -> bool:
        return (z3 is not None) and isinstance(obj, z3.z3.ModelRef)

    def relationalize(self, obj: Any, walker_func) -> Tuple[List[Atom], List[Relation]]:
        """
        Convert a z3.ModelRef into (atoms, relations).
        walker_func is unused (we recurse directly on the model), but is accepted to match the protocol.
        """
        assert self.can_handle(obj), "Z3ModelRelationalizer can only handle z3.ModelRef"
        self._reset_state()

        m: z3.ModelRef = obj

        # 1) Materialize finite universes for uninterpreted sorts (if the model provides them).
        #    This gives stable atoms for domain elements like k!0, k!1, ...
        self._materialize_universes(m)

        # 2) Create Decl atoms and Const atoms for every declaration in the model.
        for decl in m.decls():
            decl_atom = self._make_decl_atom(decl)
            # Only constants (arity 0) get a sibling Const atom (a visible handle in the diagram).
            if decl.arity() == 0:
                const_atom_id = self._const_atom_id(decl.name())
                self._add_atom(const_atom_id, "Const", decl.name())
                self._add_edge("defines", const_atom_id, decl_atom.id)

        # 3) Emit value edges and function entries.
        for decl in m.decls():
            if decl.arity() == 0:
                self._handle_constant_decl(m, decl)
            else:
                self._handle_function_decl(m, decl)

        return list(self._atoms_by_id.values()), self._relations

    # ---------- Internal state & helpers ----------

    def _reset_state(self):
        from dataclasses import dataclass  # local import to avoid confusion in snippet headers
        # Use the project’s Atom/Relation classes; we assume they’re already imported.
        self._atoms_by_id: Dict[str, Atom] = {}
        # Maps a (value-hash, sort-name) to AtomID for stable reuse
        self._val_to_atom: Dict[Any, str] = {}
        self._relations: List[Relation] = []

    # -- Atom/edge creation ----------------------------------------------------

    def _add_atom(self, id_: str, type_: str, label: str) -> Atom:
        """Create or return the existing Atom with this id."""
        if id_ in self._atoms_by_id:
            return self._atoms_by_id[id_]
        a = Atom(id=id_, type=type_, label=label)
        self._atoms_by_id[id_] = a
        return a

    def _add_edge(self, name: str, src: str, dst: str):
        """Append a new Relation edge."""
        self._relations.append(Relation(name=name, source_id=src, target_id=dst))

    # -- Universes -------------------------------------------------------------

    def _collect_candidate_sorts(self, m: "z3.ModelRef") -> List["z3.SortRef"]:
        """Collect sorts that appear in domains/ranges of all decls."""
        sorts = set()
        for d in m.decls():
            sorts.add(d.range())
            for i in range(d.arity()):
                sorts.add(d.domain(i))
        return list(sorts)

    def _is_uninterp_sort(self, s: "z3.SortRef") -> bool:
        return s.kind() == z3.Z3_UNINTERPRETED_SORT

    def _materialize_universes(self, m: "z3.ModelRef"):
        """
        Create atoms for elements of finite uninterpreted universes,
        binding Z3's elements to stable atom ids up front.
        """
        for s in self._collect_candidate_sorts(m):
            if not self._is_uninterp_sort(s):
                continue
            try:
                univ = m.get_universe(s)
            except Exception:
                univ = None
            if not univ:
                continue
            for elem in univ:
                # Ensures an atom exists and memo is populated
                _ = self._atom_of_value(elem, expected_sort=s)

    # -- Declarations & constants ---------------------------------------------

    def _decl_atom_id(self, decl: "z3.FuncDeclRef") -> str:
        return f"Decl::{decl.name()}/{decl.arity()}"

    def _const_atom_id(self, name: str) -> str:
        return f"Const::{name}"

    def _make_decl_atom(self, decl: "z3.FuncDeclRef") -> Atom:
        """Create the Decl::<name>/<arity> atom with a readable signature label."""
        id_ = self._decl_atom_id(decl)
        dom = [decl.domain(i).name() for i in range(decl.arity())]
        rng = decl.range().name()
        sig = f"{decl.name()}({', '.join(dom)}) → {rng}"
        return self._add_atom(id_, "Decl", sig)

    def _handle_constant_decl(self, m: "z3.ModelRef", decl: "z3.FuncDeclRef"):
        """
        For arity-0 declarations:
          Const::<name> --value_of--> <value-atom>
          (and array values get decomposed into map/default edges)
        """
        const_id = self._const_atom_id(decl.name())
        expected_sort = decl.range()

        # Get the value from the model (FuncDeclRef → value AstRef)
        try:
            v = m[decl]
        except Exception:
            # Fallback: evaluate the 0-ary application
            try:
                v = m.eval(decl())
            except Exception:
                v = None

        if v is None:
            return

        val_atom_id = self._atom_of_value(v, expected_sort=expected_sort)
        self._add_edge("value_of", const_id, val_atom_id)

        # If it's an array, decompose into explicit key/value/default edges for visibility.
        if self._looks_like_array_value(v):
            self._decompose_array_value(decl.name(), v, parent_const_id=const_id)

    # -- Functions -------------------------------------------------------------

    def _handle_function_decl(self, m: "z3.ModelRef", decl: "z3.FuncDeclRef"):
        """
        For arity>0: create App-nodes for each explicit entry in the model’s FuncInterp.
        Edges:
            App --of--> Decl
            App --arg_i--> <arg-atom>
            App --result--> <result-atom>
        Also, if available, add Decl --default--> <else-atom>.
        """
        decl_id = self._decl_atom_id(decl)
        interp = m[decl]

        # Typical case: z3.FuncInterpRef with explicit entries and an optional else/default.
        if isinstance(interp, z3.FuncInterpRef):
            for idx in range(interp.num_entries()):
                entry = interp.entry(idx)
                app_id = f"App::{decl.name()}/{idx}"
                self._add_atom(app_id, "App", f"{decl.name()}@{idx}")
                self._add_edge("of", app_id, decl_id)

                # Arguments are typed by the function's domain sorts
                for ai in range(entry.num_args()):
                    arg_val = entry.arg(ai)
                    arg_sort = decl.domain(ai)
                    arg_atom = self._atom_of_value(arg_val, expected_sort=arg_sort)
                    self._add_edge(f"arg_{ai}", app_id, arg_atom)

                # Result is typed by the function's range sort
                res_atom = self._atom_of_value(entry.value(), expected_sort=decl.range())
                self._add_edge("result", app_id, res_atom)

            # Default (else) result, if present
            try:
                else_v = interp.else_value()
                if else_v is not None:
                    else_atom = self._atom_of_value(else_v, expected_sort=decl.range())
                    self._add_edge("default", decl_id, else_atom)
            except Exception:
                pass

        # Rare case: not a FuncInterpRef (skip gracefully).
        else:
            return

    # -- Arrays as values ------------------------------------------------------

    def _looks_like_array_value(self, v: "z3.AstRef") -> bool:
        try:
            return v.sort().kind() == z3.Z3_ARRAY_SORT
        except Exception:
            return False

    def _decompose_array_value(
        self,
        base_name: str,
        arr: "z3.AstRef",
        parent_const_id: Optional[str] = None
    ):
        """
        Convert an array term like Store(Store(ConstArray(d), k1, v1), k2, v2) into:
          App::<base>_map/i --of--> Const::<base>
          App ... --key--> <key-atom>
          App ... --val--> <val-atom>
          and (if any) Const::<base> --<base>_default--> <default-atom>
        """
        entries: List[Tuple[z3.AstRef, z3.AstRef]] = []
        t = arr
        default_val = None

        # Walk nested Stores to collect explicit entries
        while hasattr(z3, "is_store") and z3.is_store(t):
            base = t.arg(0)
            key = t.arg(1)
            val = t.arg(2)
            entries.append((key, val))
            t = base

        # ConstArray (default) detection works via is_const_array (new) or is_k (older)
        is_const = (hasattr(z3, "is_const_array") and z3.is_const_array(t)) or (hasattr(z3, "is_k") and z3.is_k(t))
        if is_const:
            default_val = t.arg(1)

        arr_sort = arr.sort()
        key_sort = arr_sort.domain()
        val_sort = arr_sort.range()

        # Emit explicit key/value entries (reverse to keep original write order)
        for i, (k, v) in enumerate(reversed(entries)):
            app_id = f"App::{base_name}_map/{i}"
            self._add_atom(app_id, "App", f"{base_name}_map@{i}")
            if parent_const_id:
                self._add_edge("of", app_id, parent_const_id)
            key_atom = self._atom_of_value(k, expected_sort=key_sort)
            val_atom = self._atom_of_value(v, expected_sort=val_sort)
            self._add_edge("key", app_id, key_atom)
            self._add_edge("val", app_id, val_atom)

        # Default value (if any)
        if default_val is not None and parent_const_id:
            def_atom = self._atom_of_value(default_val, expected_sort=val_sort)
            self._add_edge(f"{base_name}_default", parent_const_id, def_atom)

    # -- Value → Atom (typed) --------------------------------------------------

    def _atom_of_value(self, v: "z3.AstRef", expected_sort: Optional["z3.SortRef"] = None) -> str:
        """
        Map a Z3 value to a stable Atom and return its id.
        The 'expected_sort' (the thunk/type from context) determines the Atom.type when useful
        (e.g., distinguishing Int 3 from Real 3).
        """
        key = self._value_key(v)
        if key in self._val_to_atom:
            return self._val_to_atom[key]

        actual_sort = v.sort()
        srt = expected_sort or actual_sort  # prefer contextual type info

        # Booleans
        if z3.is_true(v) or z3.is_false(v):
            label = "true" if z3.is_true(v) else "false"
            aid = f"Bool::{label}"
            self._add_atom(aid, "Bool", label)

        # Integers
        elif srt.kind() == z3.Z3_INT_SORT and (z3.is_int_value(v) or self._looks_like_int_str(v)):
            sval = self._to_int_str(v)
            aid = f"Int::{sval}"
            self._add_atom(aid, "Int", sval)

        # Reals
        elif srt.kind() == z3.Z3_REAL_SORT and (z3.is_rational_value(v) or z3.is_algebraic_value(v)):
            sval = self._pretty_real(v)
            aid = f"Real::{sval}"
            self._add_atom(aid, "Real", sval)

        # Bit-vectors
        elif srt.kind() == z3.Z3_BV_SORT and z3.is_bv_value(v):
            width = srt.size()
            sval = str(z3.BV2Int(v, is_signed=False).as_long())
            aid = f"BV[{width}]::{sval}"
            self._add_atom(aid, f"BV[{width}]", sval)

        # Strings / Sequences
        elif srt.kind() == z3.Z3_STRING_SORT:
            sval = self._escape_label(v.as_string() if hasattr(v, "as_string") else str(v))
            aid = f"String::{sval}"
            self._add_atom(aid, "String", sval)
        elif srt.kind() == z3.Z3_SEQ_SORT:
            sval = self._escape_label(str(v))
            aid = f"Seq::{sval}"
            self._add_atom(aid, "Seq", sval)

        # Uninterpreted elements (e.g., k!0)
        elif srt.kind() == z3.Z3_UNINTERPRETED_SORT:
            label = str(v)
            aid = f"U::{srt.name()}::{label}"
            self._add_atom(aid, srt.name(), label)

        # Datatypes / tuple-sorts
        elif srt.kind() == z3.Z3_DATATYPE_SORT:
            ctor_name = v.decl().name() if z3.is_app(v) else srt.name()
            label = ctor_name
            aid = f"DT::{srt.name()}::{self._stable_hash(label, v)}"
            self._add_atom(aid, srt.name(), label)
            # Optional: expose selectors as edges. (Kept minimal to avoid z3 version fragility.)
            # for each accessor sel: self._add_edge(sel.name(), aid, self._atom_of_value(sel(v), expected_sort=sel.range()))

        # Arrays: treat the array itself as an atom (entries come from decomposition elsewhere).
        elif srt.kind() == z3.Z3_ARRAY_SORT:
            label = f"Array[{srt.domain().name()}→{srt.range().name()}]"
            aid = f"Array::{self._stable_hash(label, v)}"
            self._add_atom(aid, "Array", label)

        # Fallback: use expected sort name for type and printable string for label.
        else:
            label = self._escape_label(str(v))
            aid = f"{srt.name()}::{self._stable_hash(label, v)}"
            self._add_atom(aid, srt.name(), label)

        self._val_to_atom[key] = aid
        return aid

    # -- Keying, formatting, and small utilities ------------------------------

    def _value_key(self, v: "z3.AstRef") -> Any:
        """Stable memo key for a Z3 value: prefer hash+sort-name, fallback to (str, sort-name)."""
        try:
            return (v.hash(), v.sort().name())
        except Exception:
            return (str(v), v.sort().name())

    def _looks_like_int_str(self, v: "z3.AstRef") -> bool:
        """Heuristic: if Z3 didn’t tag it as Int, but it prints like an int, allow coercion under Int thunk."""
        try:
            return str(v).lstrip("-").isdigit()
        except Exception:
            return False

    def _to_int_str(self, v: "z3.AstRef") -> str:
        """Robustly stringify an Int value."""
        try:
            return str(v.as_long())
        except Exception:
            return str(int(str(v)))

    def _pretty_real(self, v: "z3.AstRef") -> str:
        """Prefer exact rationals; fall back to a finite decimal for algebraics."""
        if z3.is_rational_value(v):
            num = v.numerator_as_long()
            den = v.denominator_as_long()
            return f"{num}/{den}" if den != 1 else f"{num}"
        try:
            return v.as_decimal(20).rstrip("?")
        except Exception:
            return str(v)

    def _escape_label(self, s: str) -> str:
        return s.replace("\n", "\\n")

    def _stable_hash(self, head: str, v: Any) -> str:
        """Short, stable-ish hash for labeling anonymous/complex values (not cryptographic)."""
        base = f"{head}|{str(v)}"
        h = 0
        for ch in base:
            h = (h * 131 + ord(ch)) & 0xFFFFFFFF
        return f"{h:08x}"


In [15]:
diagram(model)